In [ ]:
import torch
from elasticai.creator.nn.fixed_point._math_operations import MathOperations
from elasticai.creator.nn.fixed_point._two_complement_fixed_point_config import FixedPointConfig
from elasticai.creator.vhdl.testbench_helper import tensor_to_vhdl_vector
from torch import nn, Tensor

# Creator imports
from elasticai.creator.nn import Sequential
from elasticai.creator.nn.fixed_point import Linear, Tanh
from package.dnn.pytorch_handler import __model_settings_common
from package.dnn.pytorch_handler import ModelRegistry

In [ ]:
models_available = ModelRegistry()

In [ ]:
total_bits = 8
frac_bits = 5
bn_affine = True
num_steps = 2**8
interval = (-4, 3.96875)

In [ ]:
@models_available.register
class dnn_ae_v1_Q_wo_BN(__model_settings_common):
    """Class of an autoencoder with Dense-Layer for feature extraction"""
    def __init__(self, input_size=32, output_size=3):
        super().__init__('Autoencoder')
        self.model_shape = (1, input_size)
        self.model_embedded = True
        iohiddenlayer = [input_size, 20, 14, output_size]
        do_train_bias = True
        do_train_batch = True

        # --- Encoder Path
        self.encoder = Sequential(
            Linear(in_features=iohiddenlayer[0], out_features=iohiddenlayer[1], bias=do_train_bias, total_bits=total_bits, frac_bits=frac_bits, parallel=False),
            Tanh(total_bits=total_bits, frac_bits=frac_bits, num_steps=num_steps, sampling_intervall=interval),
            Linear(in_features=iohiddenlayer[1], out_features=iohiddenlayer[2], bias=do_train_bias, total_bits=total_bits, frac_bits=frac_bits, parallel=False),
            Tanh(total_bits=total_bits, frac_bits=frac_bits, num_steps=num_steps, sampling_intervall=interval),
            Linear(in_features=iohiddenlayer[2], out_features=iohiddenlayer[3], bias=do_train_bias, total_bits=total_bits, frac_bits=frac_bits, parallel=True),
        )
        # --- Decoder Path
        self.decoder = Sequential(
            Tanh(total_bits=total_bits, frac_bits=frac_bits, num_steps=num_steps),
            Linear(in_features=iohiddenlayer[3], out_features=iohiddenlayer[2], bias=do_train_bias, total_bits=total_bits, frac_bits=frac_bits, parallel=True),
            Tanh(total_bits=total_bits, frac_bits=frac_bits, num_steps=num_steps, sampling_intervall=interval),
            Linear(in_features=iohiddenlayer[2], out_features=iohiddenlayer[1], bias=do_train_bias, total_bits=total_bits, frac_bits=frac_bits, parallel=False),
            Tanh(total_bits=total_bits, frac_bits=frac_bits, num_steps=num_steps, sampling_intervall=interval),
            Linear(in_features=iohiddenlayer[1], out_features=iohiddenlayer[0], bias=do_train_bias, total_bits=total_bits, frac_bits=frac_bits, parallel=False),
        )

    def forward(self, x: Tensor) -> [Tensor, Tensor]:
        encoded = self.encoder(x)
        return encoded, self.decoder(encoded)
    
    def forward_first_layer(self, x: Tensor) -> Tensor:
        return self.encoder[0](x)
    
    def create_design(self, name):
        encoder = self.encoder.create_design(f"{name}_encoder")
        decoder = self.decoder.create_design(f"{name}_decoder")
        return encoder, decoder

In [ ]:
autoencoder = torch.load(f="/home/silas/PycharmProjects/denspp.offline/3_Python/runs/20250127_122436_train_dnn_ae_v1_Q_wo_BN_ae/model_ae_fold000_epoch0028.pth")

In [ ]:
config = FixedPointConfig(total_bits, frac_bits)
operations = MathOperations(config)

In [ ]:
def float_to_signed_int(value: float | list) -> int | list:
    if isinstance(value, list):
        return list(map(float_to_signed_int, value))
    return config.as_integer(value)

def flatten(lst):
    result = []
    for item in lst:
        if isinstance(item, list):  # Check if the item is a list
            result.extend(flatten(item))  # Recursively flatten sublists
        elif isinstance(item, (int, float)):  # Ensure it's a number
            result.append(item)
        else:
            raise TypeError(f"Unsupported data type: {type(item)} in list")
    return result

def get_weights_of_encoder(w_layers: list[int], model):
    layers = [model.encoder[x] for x in w_layers]
    
    weights_list = []

    for layer in layers:
        bias = [0] * layer.out_features if layer.bias is None else operations.quantize(layer.bias).tolist()
        weights = operations.quantize(layer.weight).tolist()
    
        weights_list.append(bias)
        weights_list.append(weights)

    weight_list = flatten(weights_list)
    return weight_list

weight_tensor = torch.tensor(get_weights_of_encoder([0,2,4], autoencoder))
tensor_to_vhdl_vector(weight_tensor, config, as_matrix=True)